In [158]:
from services.instrument import InstrumentHandler
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import numpy as np
import pandas as pd

In [159]:
# Helper Functions

def SMA(arr, n) -> pd.Series:
    return pd.Series(arr).rolling(n).mean()

def RSI(array, n):
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

def lower_band(sma, std, dev):
    return sma - std * dev

def upper_band(sma, std, dev):
    return sma + std * dev

# this is stupid
def save_stats_to_df(symbol, granularity, stats):
    df = pd.DataFrame()
    df.at[0,'symbol'] = symbol
    df.at[0,'granularity'] = granularity
    df.at[0,'strategy'] = stats['_strategy']
    df.at[0,'Start'] = stats['Start']
    df.at[0,'End'] = stats['End']
    df.at[0,'Return [%]'] = stats['Return [%]']
    df.at[0,'Buy & Hold Return [%]'] = stats['Buy & Hold Return [%]']
    df.at[0,'Sharpe Ratio'] = stats['Sharpe Ratio']
    df.at[0,'Sortino Ratio'] = stats['Sortino Ratio']
    df.at[0,'Calmar Ratio'] = stats['Calmar Ratio']
    return df

results = pd.DataFrame()

In [160]:
# Strategies declaraion

class SMA_Strategy(Strategy):
    # Default values
    l1 = 5
    l2 = 25
    
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, self.l1)
        self.ma2 = self.I(SMA, price, self.l2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()
            
class Momentum_Strategy(Strategy):
    # Default values
    d_rsi = 14
    sell_level = 70
    level = 30
    
    
    def init(self):
       
        # Compute daily RSI(14)
        self.my_rsi = self.I(RSI, self.data.Close, self.d_rsi) 
        
    def next(self):
        price = self.data.Close[-1]
        
        if (self.my_rsi[-1] < self.level):        
            self.buy()
        
        elif self.my_rsi[-1] > self.sell_level:
            self.position.close()
            
            
class Mean_Reversion_Strategy(Strategy):
    # Default values
    SMA = 30
    dev = 2
   
    
    def init(self):
        self.std = SMA(self.data.Close, self.SMA).std()
        self.ma = self.I(SMA, self.data.Close, self.SMA) 
        self.lower = self.I(lower_band, self.ma, self.std, self.dev) 
        self.upper = self.I(upper_band, self.ma, self.std, self.dev) 

    def next(self):        
        if crossover(self.data.Close, self.lower):
            self.buy()
        elif crossover(self.upper,self.data.Close ):
            self.sell()
            

# Simple Moving Average
---


## Bitcoin / USD SMA

In [161]:
btc_usd = InstrumentHandler("BTC_USD", "D", 256)
btc_usd_sma_bt = Backtest(btc_usd.get_backtest_dataset(), SMA_Strategy, commission=.002,
              exclusive_orders=True, cash=1000000)
stats = btc_usd_sma_bt.run()
btc_usd_sma_bt.plot()

BTC_USD Initialized


<ipython-input-161-661648f396be>:5: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  btc_usd_sma_bt.plot()


Row(id='19366', ...)

### Optimization

In [162]:
stats = btc_usd_sma_bt.optimize(l1=range(5, 30, 5),
                    l2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.l1 < param.l2)

results = results.append(save_stats_to_df('BTC_USD', "M1", stats=stats))

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


## Gold SMA

In [163]:
au_usd_sma = InstrumentHandler("XAU_USD", "D", 256)
au_usd_sma_bt = Backtest(au_usd_sma.get_backtest_dataset(), SMA_Strategy, commission=.002,
              exclusive_orders=True, cash=1000000)
stats = au_usd_sma_bt.run()
au_usd_sma_bt.plot()

XAU_USD Initialized


<ipython-input-163-1b7bcc0f2d97>:5: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  au_usd_sma_bt.plot()


Row(id='19965', ...)

In [164]:
stats = au_usd_sma_bt.optimize(l1=range(5, 30, 5),
                    l2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.l1 < param.l2)

results = results.append(save_stats_to_df('XAU_USD', "D", stats=stats))

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


## EU50 / USD SMA

In [120]:
eu50_eur_sma = InstrumentHandler("EU50_EUR", "D", 256)
eu50_eur_sma_bt = Backtest(eu50_eur_sma.get_backtest_dataset(), SMA_Strategy, commission=.002,
              exclusive_orders=True, cash=1000000)
stats = eu50_eur_sma_bt.run()
eu50_eur_sma_bt.plot()

EU50_EUR Initialized


<ipython-input-120-1a472660418d>:5: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  eu50_eur_sma_bt.plot()


Row(id='9328', ...)

In [165]:
stats = eu50_eur_sma_bt.optimize(l1=range(5, 30, 5),
                    l2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.l1 < param.l2)

results = results.append(save_stats_to_df('EU50_EUR', "D", stats=stats))

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


# Momentum Strategy
---

## LTC / USD Momentum

In [166]:
ltc_usd_sma = InstrumentHandler("LTC_USD", "M30", 256)
ltc_usd_sma_bt = Backtest(ltc_usd_sma.get_backtest_dataset(), Momentum_Strategy, commission=.002, exclusive_orders=True)
stats = ltc_usd_sma_bt.run()
ltc_usd_sma_bt.plot()

LTC_USD Initialized


Row(id='20612', ...)

In [167]:
stats = ltc_usd_sma_bt.optimize(
            d_rsi = range(3, 27, 3),
            sell_level = range(60, 90, 5),
            level = range(5, 30, 5),
            maximize='Equity Final [$]',
            constraint=lambda param: param.level < param.sell_level)

results = results.append(save_stats_to_df('LTC_USD', 'M5', stats))

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


## Platinum Momentum

In [168]:
xpt_usd_sma = InstrumentHandler("XPT_USD", "D", 256)
xpt_usd_sma_bt = Backtest(xpt_usd_sma.get_backtest_dataset(), Momentum_Strategy, commission=.002, exclusive_orders=True)
stats = xpt_usd_sma_bt.run()
xpt_usd_sma_bt.plot()

XPT_USD Initialized


<ipython-input-168-026ef996a5c6>:4: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  xpt_usd_sma_bt.plot()


Row(id='21307', ...)

In [169]:
stats = xpt_usd_sma_bt.optimize(
            d_rsi = range(3, 27, 3),
            sell_level = range(60, 90, 5),
            level = range(5, 30, 5),
            maximize='Equity Final [$]',
            constraint=lambda param: param.level < param.sell_level)

results = results.append(save_stats_to_df('XPT_USD', 'D', stats))

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


## Singapur 30 Momentum

In [173]:
sg30_sgd_sma = InstrumentHandler("SG30_SGD", "D", 256)
sg30_sgd_sma_bt = Backtest(sg30_sgd_sma.get_backtest_dataset(), Momentum_Strategy, commission=.002, exclusive_orders=True)
stats = sg30_sgd_sma_bt.run()
sg30_sgd_sma_bt.plot()

SG30_SGD Initialized


<ipython-input-173-3299dc585ba8>:4: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  sg30_sgd_sma_bt.plot()


Row(id='22964', ...)

In [174]:
stats = sg30_sgd_sma_bt.optimize(
            d_rsi = range(3, 27, 3),
            sell_level = range(60, 90, 5),
            level = range(5, 30, 5),
            maximize='Equity Final [$]',
            constraint=lambda param: param.level < param.sell_level)

results = results.append(save_stats_to_df('SG30_SGD', 'D', stats))

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [175]:
results

,symbol,granularity,strategy,Start,End,Return [%],Buy & Hold Return [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio
0,BTC_USD,M1,"SMA_Strategy(l1=5,l2=65)",2021-01-25 22:00:00+00:00,2022-01-20 22:00:00+00:00,88.980697,14.404254,0.991566,4.273261,5.455632
0,XAU_USD,D,"SMA_Strategy(l1=10,l2=15)",2021-01-26 22:00:00+00:00,2022-01-20 22:00:00+00:00,7.791106,-0.530784,0.745280,1.237652,1.738982
0,EU50_EUR,D,"SMA_Strategy(l1=5,l2=20)",2021-01-24 22:00:00+00:00,2022-01-20 22:00:00+00:00,8.291526,17.156053,0.670054,1.137449,1.242790
0,LTC_USD,M5,"Momentum_Strategy(d_rsi=6,sell_level=85,level=10)",2022-01-14 14:00:00+00:00,2022-01-21 21:30:00+00:00,6.561837,-21.433494,1.884363,30.164765,95.308100
0,XPT_USD,M5,"Momentum_Strategy(d_rsi=9,sell_level=80,level=15)",2021-01-04 22:00:00+00:00,2022-01-20 22:00:00+00:00,23.089023,-7.307293,1.783461,4.604959,6.583615
0,SG30_SGD,D,"Momentum_Strategy(d_rsi=3,sell_level=85,level=25)",2021-01-28 22:00:00+00:00,2022-01-20 22:00:00+00:00,2.405619,3.228371,0.256517,0.379725,0.290347


# Mean Reversion
---


## ETH / USD

In [182]:
eth_usd_mr = InstrumentHandler("ETH_USD", "D", 256)
eth_usd_mr_bt = Backtest(eth_usd_mr.get_backtest_dataset(), Mean_Reversion_Strategy, commission=.002,
              exclusive_orders=True, cash=1000000)
stats = eth_usd_mr_bt.run()
eth_usd_mr_bt.plot()

ETH_USD Initialized


<ipython-input-182-9084d926b46a>:5: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  eth_usd_mr_bt.plot()


Row(id='25376', ...)

In [181]:
stats = eth_usd_mr_bt.optimize(
    dev = [x * 0.1 for x in range(0, 30)],
    SMA = range(5, 30, 3),
    maximize='Equity Final [$]')

results = results.append(save_stats_to_df('ETH_USD', 'D', stats))

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


## Silver Mean Reversion

In [183]:
xag_usd_mr = InstrumentHandler("XAG_USD", "D", 256)
xag_usd_mr_bt = Backtest(xag_usd_mr.get_backtest_dataset(), Mean_Reversion_Strategy, commission=.002,
              exclusive_orders=True, cash=1000000)
stats = xag_usd_mr_bt.run()
xag_usd_mr_bt.plot()


XAG_USD Initialized


<ipython-input-183-b9996d6e7e33>:5: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  xag_usd_mr_bt.plot()


Row(id='25867', ...)

In [184]:
stats = xag_usd_mr_bt.optimize(
    dev = [x * 0.1 for x in range(0, 30)],
    SMA = range(5, 30, 3),
    maximize='Equity Final [$]')

results = results.append(save_stats_to_df('XAG_USD', 'D', stats))

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [185]:
results

,symbol,granularity,strategy,Start,End,Return [%],Buy & Hold Return [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio
0,BTC_USD,M1,"SMA_Strategy(l1=5,l2=65)",2021-01-25 22:00:00+00:00,2022-01-20 22:00:00+00:00,88.980697,14.404254,0.991566,4.273261,5.455632
0,XAU_USD,D,"SMA_Strategy(l1=10,l2=15)",2021-01-26 22:00:00+00:00,2022-01-20 22:00:00+00:00,7.791106,-0.530784,0.745280,1.237652,1.738982
0,EU50_EUR,D,"SMA_Strategy(l1=5,l2=20)",2021-01-24 22:00:00+00:00,2022-01-20 22:00:00+00:00,8.291526,17.156053,0.670054,1.137449,1.242790
0,LTC_USD,M5,"Momentum_Strategy(d_rsi=6,sell_level=85,level=10)",2022-01-14 14:00:00+00:00,2022-01-21 21:30:00+00:00,6.561837,-21.433494,1.884363,30.164765,95.308100
0,XPT_USD,M5,"Momentum_Strategy(d_rsi=9,sell_level=80,level=15)",2021-01-04 22:00:00+00:00,2022-01-20 22:00:00+00:00,23.089023,-7.307293,1.783461,4.604959,6.583615
0,SG30_SGD,D,"Momentum_Strategy(d_rsi=3,sell_level=85,level=25)",2021-01-28 22:00:00+00:00,2022-01-20 22:00:00+00:00,2.405619,3.228371,0.256517,0.379725,0.290347
0,ETH_USD,D,"Mean_Reversion_Strategy(dev=0.5,SMA=5)",2021-03-21 21:00:00+00:00,2022-01-20 22:00:00+00:00,111.803776,47.091294,0.619697,3.572255,4.498140
0,xag_USD,D,"Mean_Reversion_Strategy(dev=0.1,SMA=29)",2021-01-26 22:00:00+00:00,2022-01-20 22:00:00+00:00,35.592694,-3.824693,1.568007,4.028571,3.935541


## Australia 200 Mean Reversion

In [207]:
au200_aud_mr = InstrumentHandler("AU200_AUD", "D", 256)
au200_aud_mr_bt = Backtest(au200_aud_mr.get_backtest_dataset(), Mean_Reversion_Strategy, commission=.002, exclusive_orders=True)
stats = au200_aud_mr_bt.run()
au200_aud_mr_bt.plot()


AU200_AUD Initialized


<ipython-input-207-0e95acb3c9b2>:4: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  au200_aud_mr_bt.plot()


Row(id='27456', ...)

In [209]:
stats = au200_aud_mr_bt.optimize(
    dev = [x * 0.1 for x in range(0, 30)],
    SMA = range(5, 30, 3),
    maximize='Equity Final [$]')


/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [210]:

results = results.append(save_stats_to_df('AU200_AUD', 'D', stats))

In [211]:
results

,symbol,granularity,strategy,Start,End,Return [%],Buy & Hold Return [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio
0,BTC_USD,M1,"SMA_Strategy(l1=5,l2=65)",2021-01-25 22:00:00+00:00,2022-01-20 22:00:00+00:00,88.980697,14.404254,0.991566,4.273261,5.455632
0,XAU_USD,D,"SMA_Strategy(l1=10,l2=15)",2021-01-26 22:00:00+00:00,2022-01-20 22:00:00+00:00,7.791106,-0.530784,0.745280,1.237652,1.738982
0,EU50_EUR,D,"SMA_Strategy(l1=5,l2=20)",2021-01-24 22:00:00+00:00,2022-01-20 22:00:00+00:00,8.291526,17.156053,0.670054,1.137449,1.242790
0,LTC_USD,M5,"Momentum_Strategy(d_rsi=6,sell_level=85,level=10)",2022-01-14 14:00:00+00:00,2022-01-21 21:30:00+00:00,6.561837,-21.433494,1.884363,30.164765,95.308100
0,XPT_USD,M5,"Momentum_Strategy(d_rsi=9,sell_level=80,level=15)",2021-01-04 22:00:00+00:00,2022-01-20 22:00:00+00:00,23.089023,-7.307293,1.783461,4.604959,6.583615
0,SG30_SGD,D,"Momentum_Strategy(d_rsi=3,sell_level=85,level=25)",2021-01-28 22:00:00+00:00,2022-01-20 22:00:00+00:00,2.405619,3.228371,0.256517,0.379725,0.290347
0,ETH_USD,D,"Mean_Reversion_Strategy(dev=0.5,SMA=5)",2021-03-21 21:00:00+00:00,2022-01-20 22:00:00+00:00,111.803776,47.091294,0.619697,3.572255,4.498140
0,xag_USD,D,"Mean_Reversion_Strategy(dev=0.1,SMA=29)",2021-01-26 22:00:00+00:00,2022-01-20 22:00:00+00:00,35.592694,-3.824693,1.568007,4.028571,3.935541
0,FR40_EUR,D,Mean_Reversion_Strategy,2021-01-26 22:00:00+00:00,2022-01-20 22:00:00+00:00,0.000000,29.215559,NaN,NaN,NaN
0,FR40_EUR,D,"Mean_Reversion_Strategy(dev=0.0,SMA=5)",2021-01-28 22:00:00+00:00,2022-01-20 22:00:00+00:00,0.000000,-14.725202,NaN,NaN,NaN
